In [1]:
import mlflow
mlflow.autolog()

In [2]:
# @title Importation données
import pandas as pd
ct = 'QueryResults (1).csv'
df = pd.read_csv(ct)
df

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,20485075,How do I define multiple conditions on a join ...,"<p>Ok, lets say I have order and items tables....",<java><jpa><playframework><playframework-2.0><...,1,1984,2
1,20485079,What is the equivalent of a C# gridview in jav...,"<p>I am trying to learn java, I am working on ...",<c#><java><asp.net><swing><gridview>,2,12333,1
2,20485143,"Listview cursor adapter, always get first item...",<p>I have this piece of code for listview usin...,<android><listview><android-listview><simplecu...,1,1075,2
3,20485144,OpenSSL RSA-2048 unencrypted block is longer t...,<p>I am using the OpenSSL library in order to ...,<c><encryption><cryptography><openssl><rsa>,0,527,1
4,20485176,Returning char into main method,<p>The goal of my program is kind of like whee...,<java><eclipse><string><methods><char>,0,1278,3
...,...,...,...,...,...,...,...
49995,22814994,StructureMap use specific instance of type per...,"<p>Is there a way, using StructureMap (the Dep...",<c#><asp.net><dependency-injection><inversion-...,0,369,2
49996,22815009,Add a reference column migration in Rails 4,<p>A user has many uploads. I want to add a co...,<ruby-on-rails><ruby-on-rails-4><foreign-keys>...,368,397557,8
49997,22815010,Hibernate mapping with group by clause?,<p>I have <code>@OneToMany</code> mapping in a...,<java><sql><hibernate><group-by><hibernate-map...,0,1755,1
49998,22815022,java.lang.NoClassDefFoundError: android.suppor...,<p>I have a class that extends <code>android.s...,<java><android><eclipse><android-fragments><an...,0,1441,1


In [3]:
# @title Filtre des tags

df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3]

/var/folders/_l/cjjst_c50lx2nkkr280zbp440000gp/T/ipykernel_999/3543793987.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nb_tags'] = df['Tags'].apply(lambda x: len(x))


In [4]:
df['Tags']

1              [c#, java, asp.net]
16             [php, mysql, linux]
20           [swift, xcode, macos]
23         [javascript, php, html]
26                 [c#, c, string]
                   ...            
49981    [c#, iphone, objective-c]
49985          [php, jquery, html]
49989    [javascript, jquery, css]
49992     [c#, asp.net, angularjs]
49994           [html, css, swift]
Name: Tags, Length: 6644, dtype: object

In [5]:
# @title Nettoyage du texte
import pickle
import nltk
nltk.download('word_tokenize')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

custom_punkt_path = '/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/punkt'
punkt_path = nltk.data.find(f'{custom_punkt_path}/english.pickle')

with open(punkt_path, 'rb') as file:
    punkt_model = pickle.load(file)
nltk.data.path.append(custom_punkt_path)

with open('/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/corpora/stopwords/english', 'r') as file:
    custom_stopwords = file.read().splitlines()
    
# Nettoyage
def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    #word_tokens = word_tokenize(sentence_clean, language='english', path_to_punkt=punkt_path)
    word_tokens = punkt_model.tokenize(sentence_clean)
    return word_tokens


stop_w = custom_stopwords + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']
def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


2023/12/14 21:02:17 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
[nltk_data] Error loading word_tokenize: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self signed certificate in certificate chain
[nltk_data]     (_ssl.c:997)>


In [6]:
# @title Encoding TF - IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténez les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

In [7]:
# @title Modélisation  LDA 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#df_pca_scaled = pd.DataFrame(scaler.fit_transform(df_pca), columns=df_pca.columns)

X = tfidf_df
#X = df_pca_scaled
# Création du modèle LDA avec 50 sujets
num_topics = 50
with mlflow.start_run(run_name='unsupervised') as run:
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)
    mlflow.sklearn.log_model(
        sk_model=lda_model,
        artifact_path="model",
        registered_model_name="unsupervised",
    )
# Obtenir les trois sujets les plus probables pour chaque ligne
top_n = 3
top_topics = lda_model.transform(X).argsort(axis=1)[:, -top_n:]

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [topics for topics in top_topics]

2023/12/14 21:02:21 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2023/12/14 21:02:43 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/12/14 21:02:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/12/14 21:02:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/12/14 21:02:48 WARNING mlflow.sklearn: Model was missing function: predict. No

In [8]:
"""
import gensim
import numpy as np

# Pré-traitement du texte
processed_text = gensim.utils.simple_preprocess(df['Cleaned_Body'])

# Conversion de la série en chaîne
processed_text_str = processed_text.to_string()

# Création du modèle LDA avec 50 sujets
num_topics = 50
lda_model_gensim = gensim.models.ldamodel.LdaModel(
    processed_text_str, num_topics=num_topics, random_state=42
)

# Obtenir les trois sujets les plus probables pour chaque ligne
top_n = 3
top_topics = lda_model_gensim.get_topics(processed_text_str, topn=top_n)

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [
    [topic[0] for topic in topics] for topics in top_topics
]
"""

"\nimport gensim\nimport numpy as np\n\n# Pré-traitement du texte\nprocessed_text = gensim.utils.simple_preprocess(df['Cleaned_Body'])\n\n# Conversion de la série en chaîne\nprocessed_text_str = processed_text.to_string()\n\n# Création du modèle LDA avec 50 sujets\nnum_topics = 50\nlda_model_gensim = gensim.models.ldamodel.LdaModel(\n    processed_text_str, num_topics=num_topics, random_state=42\n)\n\n# Obtenir les trois sujets les plus probables pour chaque ligne\ntop_n = 3\ntop_topics = lda_model_gensim.get_topics(processed_text_str, topn=top_n)\n\n# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables\ndf['lda_predict'] = [\n    [topic[0] for topic in topics] for topics in top_topics\n]\n"

In [9]:
"""
# @title LDAvis
import pyLDAvis
#import pyLDAvis.sklearn

pyLDAvis_data = pyLDAvis.prepare(lda_model, X)

pyLDAvis.display(pyLDAvis_data)
"""

'\n# @title LDAvis\nimport pyLDAvis\n#import pyLDAvis.sklearn\n\npyLDAvis_data = pyLDAvis.prepare(lda_model, X)\n\npyLDAvis.display(pyLDAvis_data)\n'

In [10]:
df[['Tags','lda_predict']]

,Tags,lda_predict
0,"[c#, java, asp.net]","[49, 31, 28]"
1,"[php, mysql, linux]","[12, 49, 31]"
2,"[swift, xcode, macos]","[39, 11, 31]"
3,"[javascript, php, html]","[12, 49, 28]"
4,"[c#, c, string]","[49, 39, 28]"
...,...,...
6639,"[c#, iphone, objective-c]","[49, 45, 28]"
6640,"[php, jquery, html]","[49, 31, 8]"
6641,"[javascript, jquery, css]","[49, 10, 28]"
6642,"[c#, asp.net, angularjs]","[25, 32, 28]"


In [11]:
from collections import defaultdict

# Création d'un dictionnaire pour compter les correspondances
tag_counts = defaultdict(lambda: defaultdict(int))

# Parcours des lignes pour compter les correspondances entre les tags et les numéros
for index, row in df.iterrows():
    tags = row['Tags']
    lda_topics = row['lda_predict']

    # Comptage des correspondances entre les tags et les numéros
    for topic in lda_topics:
        for tag in tags:
            tag_counts[topic][tag] += 1

# Convertir le dictionnaire en DataFrame
results = pd.DataFrame(tag_counts).T
results.fillna(0, inplace=True)
results = results.rename_axis('LDA Topic').reset_index()

In [12]:

topic_to_tag = {}
tag_to_topic = {}

# Parcourir les lignes du DataFrame et associer chaque sujet au tag le plus fréquent
for _, row in results.iterrows():
    topic = row['LDA Topic']
    tags_except_lda = row.drop('LDA Topic')
    
    # Vérifier si le tag a déjà été associé à un autre sujet
    if topic not in topic_to_tag and not tags_except_lda.empty:
        # Si le tag n'a pas encore été associé à un sujet
        most_common_tag = tags_except_lda.idxmax()
        
        # Vérifier si le tag a déjà été associé à un autre sujet
        while most_common_tag in tag_to_topic:
            # Si le tag a déjà été associé, trouver le prochain tag le plus fréquent
            tags_except_lda = tags_except_lda.drop(most_common_tag)
            if tags_except_lda.empty:
                # Si tous les tags ont déjà été associés, sortir de la boucle
                break
            most_common_tag = tags_except_lda.idxmax()
        
        # Associer le sujet au tag le plus fréquent
        topic_to_tag[topic] = most_common_tag
        tag_to_topic[most_common_tag] = topic
# Afficher les associations sujet-tag
print("Associations sujet-tag :")
print(topic_to_tag)

# Remplacer les numéros dans df['lda_predict'] par les tags
df['lda_predict'] = df['lda_predict'].apply(lambda topics: [topic_to_tag.get(t, 'Unknown') for t in topics])


Associations sujet-tag :
{49.0: 'javascript', 31.0: 'c#', 28.0: 'html', 12.0: '.net', 39.0: 'string', 11.0: 'ios', 45.0: 'objective-c', 23.0: 'node.js', 48.0: 'jquery', 8.0: 'forms', 5.0: 'asp.net', 10.0: 'css', 2.0: 'python', 32.0: 'swift', 35.0: 'java', 14.0: 'iphone', 15.0: 'spring', 37.0: 'pandas', 27.0: 'android', 41.0: 'c++', 38.0: 'mysql', 3.0: 'spring-boot', 25.0: 'multithreading', 21.0: 'php', 16.0: 'json', 30.0: 'c', 18.0: 'arrays', 26.0: 'angularjs', 22.0: 'ajax', 42.0: 'performance', 17.0: 'xcode', 36.0: 'database', 0.0: 'sql', 46.0: 'ruby-on-rails', 34.0: 'windows', 13.0: 'ruby', 19.0: 'sql-server', 29.0: 'macos', 33.0: 'image', 24.0: 'regex', 40.0: 'xml', 47.0: 'asp.net-mvc', 6.0: 'linux', 1.0: 'angular', 7.0: 'python-3.x', 4.0: 'django', 44.0: 'wpf', 9.0: 'wordpress', 43.0: 'reactjs'}


In [14]:
import json
json_data = json.dumps(topic_to_tag)
with open("topic_to_tag.json", "w") as file:
    file.write(json_data)
mlflow.log_artifact("topic_to_tag.json")

import os
os.remove("topic_to_tag.json")

In [25]:
df[['Tags','lda_predict']]

,Tags,lda_predict
0,"[c#, java, asp.net]","[javascript, c#, html]"
1,"[php, mysql, linux]","[.net, javascript, c#]"
2,"[swift, xcode, macos]","[string, ios, c#]"
3,"[javascript, php, html]","[.net, javascript, html]"
4,"[c#, c, string]","[javascript, string, html]"
...,...,...
6639,"[c#, iphone, objective-c]","[javascript, objective-c, html]"
6640,"[php, jquery, html]","[javascript, c#, forms]"
6641,"[javascript, jquery, css]","[javascript, css, html]"
6642,"[c#, asp.net, angularjs]","[multithreading, swift, html]"


In [26]:
# @title Scoring  LDA personalisé
gg_scores = []

for index, row in df.iterrows():
    real_tags = set(row['Tags'])
    lda_tags = set(row['lda_predict'])

    common_tags = real_tags.intersection(lda_tags)  # Tags communs entre les deux ensembles

    if len(common_tags) == 3:
        gg_scores.append(1)
    elif len(common_tags) == 2:
        gg_scores.append(2/3)
    elif len(common_tags) == 1:
        gg_scores.append(1/3)
    else:
        gg_scores.append(0)

# Calcul de la moyenne des scores "good guess"
gg = sum(gg_scores) / len(gg_scores)
print("gg = ", round(gg,2)*100," %")

gg =  25.0  %


In [28]:
# @title Scoring  LDA coherence score et perplexity
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# Transformer le corpus TF-IDF en une liste de listes de mots
tokenized_data = [text.split() for text in df['Cleaned_Body']]

# Créer un dictionnaire Gensim à partir de vos données tokenizées
dico = Dictionary(tokenized_data)

# Calculer la cohérence
#coherence_model = CoherenceModel(model=lda_model, texts=tokenized_data, dictionary=dico, coherence='c_v')
#coherence_score = coherence_model.get_coherence()
#print("Coherence Score =",coherence_score)

perplexity = lda_model.perplexity(tfidf_df)
print("Perplexity= " ,round(perplexity,0))

Perplexity=  2437.0


In [41]:
tfidf_df

,00,000,01,02,03,04,05,06,07,08,...,wrong,www,xcode,xml,xmlns,year,yes,you,your,zip
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.084993,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.029521,0.043098,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033612,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.114930,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
6640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
6641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.073282,0.000000,0.0
6642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [32]:
#Réduction de dimension

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


pca = PCA(n_components=5)
tfidf_pca = pca.fit_transform(tfidf_df)
df_pca = pd.DataFrame(data=tfidf_pca, columns=['PC1', 'PC2','PC3','PC4','PC5'])
#print(df_pca.head())


df_pca_scaled = pd.DataFrame(scaler.fit_transform(df_pca), columns=df_pca.columns)
X = df_pca_scaled
with mlflow.start_run(run_name='unsupervised_PCA') as run:
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(X)
    mlflow.sklearn.log_model(
        sk_model=lda_model,
        artifact_path="model",
        registered_model_name="unsupervised",
    )
# Obtenir les trois sujets les plus probables pour chaque ligne
top_n = 3
top_topics = lda_model.transform(X).argsort(axis=1)[:, -top_n:]

# Créer une colonne 'lda_predict' avec la liste des sujets les plus probables
df['lda_predict'] = [topics for topics in top_topics]

gg_scores = []

for index, row in df.iterrows():
    real_tags = set(row['Tags'])
    lda_tags = set(row['lda_predict'])

    common_tags = real_tags.intersection(lda_tags)  # Tags communs entre les deux ensembles

    if len(common_tags) == 3:
        gg_scores.append(1)
    elif len(common_tags) == 2:
        gg_scores.append(2/3)
    elif len(common_tags) == 1:
        gg_scores.append(1/3)
    else:
        gg_scores.append(0)

# Calcul de la moyenne des scores "good guess"
gg = sum(gg_scores) / len(gg_scores)
print("gg = ", round(gg,2)*100," %")

perplexity = lda_model.perplexity(X)
print("Perplexity= " ,round(perplexity,0))

2023/12/08 14:25:30 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0a731fb627774beeba27e4faea97c868', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2023/12/08 14:25:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2023/12/08 14:25:32 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/12/08 14:25:32 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/12/08 14:25:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/orphila_adjovi/PJT5_Open_classrooms_MLE/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2023/12/08 14:25:44 WARNING mlflow.sklearn: Train

gg =  0.0  %
Perplexity=  76.0
